**Set environment**

In [11]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [12]:
ls -1 ${FD_RES}/results/region

annotation_ccre
annotation_chipseq_histone
annotation_chipseq_tf
annotation_chipseq_tf_subset
annotation_chromHMM
annotation_crispri_growth
annotation_crispri_hcrff
annotation_enhancer_junke
annotation_enhancer_zscore_junke
annotation_gencode
annotation_hic
annotation_tss_pol2
annotation_tss_pol2_span
hic_intact_ENCSR479XDG
KS91_K562_ASTARRseq_peak_macs_input
region_screened_crispri_flowfish.bed
region_screened_selected.bed
region_screened_selected_perbase.bed
region_screened_selected_perbase.bed.gz
region_screened_tmpra.bed
region_screened_tmpra_crispr.bed


In [13]:
ls -1 ${FD_RES}/results/region/annotation_enhancer_zscore_junke

description.tsv
enhancer_zscore_junke.concat.STARR.bed.gz
enhancer_zscore_junke.concat.STARRMPRA.bed.gz
enhancer_zscore_junke.merge.STARR.bed.gz
enhancer_zscore_junke.merge.STARRMPRA.bed.gz
enhancer_zscore_junke.peak.ASTARR.bed.gz
enhancer_zscore_junke.peak.LMPRA.bed.gz
enhancer_zscore_junke.peak.TMPRA_GATAMYC.bed.gz
enhancer_zscore_junke.peak.WSTARR.bed.gz


In [14]:
ls -1 ${FD_RES}/results/region/annotation_enhancer_zscore_junke/{*peak*bed.gz,*merge*bed.gz}

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.merge.STARR.bed.gz
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.merge.STARRMPRA.bed.gz
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.peak.ASTARR.bed.gz
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.peak.LMPRA.bed.gz
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.peak.TMPRA_GATAMYC.bed.gz
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.peak.WSTARR.bed.gz


In [15]:
FDIRY=${FD_RES}/results/region/annotation_enhancer_zscore_junke
FNAME=enhancer_zscore_junke.peak.ASTARR.bed.gz
FPATH=${FDIRY}/${FNAME}
zcat ${FPATH} | head

chr1	1013020	1013470	peak_both_1	ASTARR
chr1	1724540	1724700	peak_both_2	ASTARR
chr1	2290700	2291030	peak_both_3	ASTARR
chr1	3313260	3313400	peak_both_4	ASTARR
chr1	5856530	5856630	peak_both_5	ASTARR
chr1	5917710	5917930	peak_both_6	ASTARR
chr1	6949090	6949200	peak_both_7	ASTARR
chr1	8211980	8212260	peak_both_8	ASTARR
chr1	8873640	8873890	peak_both_9	ASTARR
chr1	8877970	8878130	peak_both_10	ASTARR


In [16]:
ls -1 ${FD_RES}/results/region/hic_intact_ENCSR479XDG/loop

annotation_enhancer_zscore_junke
annotation_tss_pol2
loopA.bed.gz
loopB.bed.gz
loop.tsv
summary


**Script**

In [17]:
cat > run_annot_array.sh << 'EOF'
#!/bin/bash

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### setup input and output
source ../config/config_duke.sh
PREFIX=$1
FD_REG=$2
FD_ANT=$3
FD_OUT=$4

FP_REG=${FD_REG}/${PREFIX}.bed.gz

FP_ANTS=($(ls ${FD_ANT}/*.bed.gz))
FP_ANT=${FP_ANTS[SLURM_ARRAY_TASK_ID]}
FN_ANT=$(basename ${FP_ANT})

FN_OUT=${PREFIX}.annotation.${FN_ANT}
FP_OUT=${FD_OUT}/${FN_OUT}

### show I/O file
echo "Input: " ${FP_REG}
echo
echo "show first few lines of input"
zcat ${FP_REG} | head -5
echo
echo "Input: " ${FP_ANT}
echo
echo "show first few lines of input"
zcat ${FP_ANT} | head -5
echo

### execute: annotation using intersect
bedtools intersect \
    -a ${FP_REG} \
    -b ${FP_ANT} \
    -wo \
    -f 0.7 \
    -F 0.7 \
    -e \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

chmod +x run_annot_array.sh

**RUN**

In [24]:
FD_REG=${FD_RES}/results/region/hic_intact_ENCSR479XDG/loop
FD_ANT=${FD_RES}/results/region/annotation_enhancer_zscore_junke
FD_OUT=${FD_RES}/results/region/hic_intact_ENCSR479XDG/loop/annotation_enhancer_zscore_junke

for PREFIX in "loopA" "loopB"; do
    echo ${PREFIX}
    
    sbatch -p ${NODE} \
        --exclude=dl-01 \
        --cpus-per-task 8 \
        --mem 4G \
        --output ${FD_LOG}/annotation_hic_intact_ENCSR479XDG_${PREFIX}_enhancer_zscore_junke.%a.txt \
        --array 0-7 \
        ./run_annot_array.sh ${PREFIX} ${FD_REG} ${FD_ANT} ${FD_OUT}
done

loopA
Submitted batch job 29915551
loopB
Submitted batch job 29915552


**Check Results**

In [25]:
cat ${FD_LOG}/annotation_hic_intact_ENCSR479XDG_loopA_enhancer_zscore_junke.0.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         05-26-23+14:24:49

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/loopA.bed.gz

show first few lines of input
chr1	770000	780000
chr1	904000	906000
chr1	920000	925000
chr1	980000	990000
chr1	984000	985000

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.concat.STARR.bed.gz

show first few lines of input
chr1	778420	779050	WSTARR
chr1	827430	827560	WSTARR
chr1	959270	959620	WSTARR
chr1	997730	997950	WSTARR
chr1	1000690	1000990	WSTARR


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/annotation_enhancer_zscore_junke/loopA.annotation.enhancer_zscore_junke.concat.STARR.bed.gz

show first few lines of output:
chr1	770000	780000	chr1	778420	779050	WSTARR	630
chr1	1000000	1005000	chr1	1000690	1000990	WSTARR	300
chr1	1000

In [26]:
cat ${FD_LOG}/annotation_hic_intact_ENCSR479XDG_loopA_enhancer_zscore_junke.7.txt

Hostname:           x1-02-2.genome.duke.edu
Slurm Array Index:  7
Time Stamp:         05-26-23+14:24:49

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/loopA.bed.gz

show first few lines of input
chr1	770000	780000
chr1	904000	906000
chr1	920000	925000
chr1	980000	990000
chr1	984000	985000

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.peak.WSTARR.bed.gz

show first few lines of input
chr1	778420	779050	WSTARR
chr1	827430	827560	WSTARR
chr1	959270	959620	WSTARR
chr1	997730	997950	WSTARR
chr1	1000690	1000990	WSTARR


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/annotation_enhancer_zscore_junke/loopA.annotation.enhancer_zscore_junke.peak.WSTARR.bed.gz

show first few lines of output:
chr1	770000	780000	chr1	778420	779050	WSTARR	630
chr1	1000000	1005000	chr1	1000690	1000990	WSTARR	300
chr1	100000

In [27]:
cat ${FD_LOG}/annotation_hic_intact_ENCSR479XDG_loopB_enhancer_zscore_junke.0.txt

Hostname:           x1-02-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         05-26-23+14:24:49

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/loopB.bed.gz

show first few lines of input
chr1	840000	850000
chr1	976000	978000
chr1	1060000	1065000
chr1	1060000	1065000
chr1	1060000	1065000

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.concat.STARR.bed.gz

show first few lines of input
chr1	778420	779050	WSTARR
chr1	827430	827560	WSTARR
chr1	959270	959620	WSTARR
chr1	997730	997950	WSTARR
chr1	1000690	1000990	WSTARR


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/annotation_enhancer_zscore_junke/loopB.annotation.enhancer_zscore_junke.concat.STARR.bed.gz

show first few lines of output:
chr1	1060000	1065000	chr1	1064070	1064300	WSTARR	230
chr1	1060000	1065000	chr1	1064510	1064750	WSTARR	240

In [28]:
cat ${FD_LOG}/annotation_hic_intact_ENCSR479XDG_loopB_enhancer_zscore_junke.1.txt

Hostname:           x1-02-1.genome.duke.edu
Slurm Array Index:  1
Time Stamp:         05-26-23+14:24:49

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/loopB.bed.gz

show first few lines of input
chr1	840000	850000
chr1	976000	978000
chr1	1060000	1065000
chr1	1060000	1065000
chr1	1060000	1065000

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.concat.STARRMPRA.bed.gz

show first few lines of input
chr1	778420	779050	WSTARR
chr1	827430	827560	WSTARR
chr1	904350	904550	LMPRA
chr1	941699	941899	LMPRA
chr1	959200	959400	LMPRA


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/annotation_enhancer_zscore_junke/loopB.annotation.enhancer_zscore_junke.concat.STARRMPRA.bed.gz

show first few lines of output:
chr1	1060000	1065000	chr1	1064070	1064300	WSTARR	230
chr1	1060000	1065000	chr1	1064510	1064750	WSTARR	

In [29]:
cat ${FD_LOG}/annotation_hic_intact_ENCSR479XDG_loopB_enhancer_zscore_junke.7.txt

Hostname:           x1-03-1.genome.duke.edu
Slurm Array Index:  7
Time Stamp:         05-26-23+14:24:49

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/loopB.bed.gz

show first few lines of input
chr1	840000	850000
chr1	976000	978000
chr1	1060000	1065000
chr1	1060000	1065000
chr1	1060000	1065000

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.peak.WSTARR.bed.gz

show first few lines of input
chr1	778420	779050	WSTARR
chr1	827430	827560	WSTARR
chr1	959270	959620	WSTARR
chr1	997730	997950	WSTARR
chr1	1000690	1000990	WSTARR


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/annotation_enhancer_zscore_junke/loopB.annotation.enhancer_zscore_junke.peak.WSTARR.bed.gz

show first few lines of output:
chr1	1060000	1065000	chr1	1064070	1064300	WSTARR	230
chr1	1060000	1065000	chr1	1064510	1064750	WSTARR	240
c